# Spectral Fit with systematics

Given a set of 1-dimensional PDFs, we would like to fit the number of events
of each type in the dataset, knowing their distributions _a priori_.

In [ ]:
push!(LOAD_PATH, "../src/")
using Batman
using DataFrames
using PyPlot

In [ ]:
# Create a fake data set (stored in a dataframe)
function generate_dataset()
    norm = [150, 300, 40]
    mvs = [20.0, 23.0, 18.0] # mean values
    svs = [2.0, 5.0, 1.0]    # sigma values

    df1 = DataFrame(energy=( randn(norm[1]).*svs[1] .+ mvs[1] ) )
    append!(df1, DataFrame(energy=( randn(norm[2]).*svs[2] .+ mvs[2] ) ))
    append!(df1, DataFrame(energy=( randn(norm[3]).*svs[3] .+ mvs[3] ) ))

    # Stitch the dataset into the module
    add_dataset(:tri, df1 )
    Batman.tri
end

# Visualize the total dataset
data = generate_dataset()
plt.hist(data.energy, bins=collect(1:0.5:40))
plt.show()

In [ ]:
# Put together a custom model to fit to the generated dataset
model = CustomModel()

@addfunction shape(μ, σ) = @. exp(-(tri.energy-μ)^2/2/σ^2)/sqrt(2π*σ^2)

## Free parameters
s  = Parameter("signal"; init=100.0)
b1 = Parameter("bkg1"; init=100.0)
b2 = Parameter("bkg2"; init=100.0)
## Data counts
observation_count = Parameter("obs"; init=Float64( size(data,1) ), constant=true )
## Known Means
sm  = Constant("Signal Mean", 20.0)
bm1 = Constant("Bkg1 Mean", 23.0)
bm2 = Constant("Bkg2 Mean", 18.0)
## Known Sigma
ss  = Constant("Signal Sig", 2.0)
bs1 = Constant("Bkg1 Sig", 5.0)
bs2 = Constant("Bkg2 Sig", 1.0)

## Build the PDF functions
@addfunction spectralpdf(b1, b2, b3, m1, m2, m3, s1, s2, s3) = begin
    -sum(log.( b1*shape(m1, s1) .+ b2*shape(m2, s2) .+ b3*shape(m3, s3) ))
end
@addfunction logextend(n, x...) = begin
    λ = sum([x...])
    λ + n*log(n) - n
end
## Construct NLL
ob1 = NLogPDF("spectralpdf", s, b1, b2, sm, bm1, bm2, ss, bs1, bs2)
ob2 = NLogPDF("logextend", observation_count, s, b1, b2)

## Include the above in our model
add_parameters!(model, [s, b1, b2, observation_count, sm, bm1, bm2, ss, bs1, bs2])
add_nlogpdfs!(model, [ob1, ob2])
@show model.lower_bounds
results = minimize!(model)

compute_profiled_uncertainties!(results; σ=1)
pretty_results(results)

In [ ]:
a = getparam(results.model, "bkg2")
x = a.likelihood_x
y = a.likelihood_y
plt.plot(x, y)
plt.show()

In [ ]:
## Given that the model is ready, we could run it over many datasets
trials = 3

fit_set = []
for i = 1:trials
    generate_dataset()
    iter_results = minimize!(model)
    push!(fit_set, iter_results.min_parameters)
end

@show fit_set

In [ ]:
correlation_plots2(results)